# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import CSV file of Output Data
file_to_load = "cities.csv"

In [3]:
#Read the file and store into a dataframe
city_df = pd.read_csv(file_to_load)
city_df = pd.DataFrame(city_df)
city_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [8]:
cities = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
vacay_df = city_df[(city_df['Max Temp'] >= 70) & (city_df['Max Temp'] <= 80)]
vacay_df = vacay_df.drop(['Date'], axis=1)
vacay_df = vacay_df[vacay_df['Wind Speed'] <= 10]
vacay_df = vacay_df[vacay_df['Humidity'] <= 55]
hotel_df = vacay_df
hotel_df

,City_ID,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,23,-18.06,13.84,72.06,5.44
153,153,porto novo,20,BR,53,-23.68,-45.44,77.00,4.70
176,176,nador,0,MA,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,51,22.92,96.51,72.24,2.77
311,311,balkanabat,99,TM,41,39.51,54.36,79.80,6.69
393,393,juiz de fora,20,BR,44,-21.76,-43.35,75.20,8.05
443,443,lucapa,65,AO,48,-8.42,20.74,78.36,2.73
444,444,birjand,0,IR,26,32.86,59.22,71.60,9.17
468,468,san cristobal,75,EC,37,-0.39,-78.55,71.60,4.70
534,534,marzuq,84,YE,36,14.40,46.47,79.44,6.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,23,-18.06,13.84,72.06,5.44,
153,153,porto novo,20,BR,53,-23.68,-45.44,77.00,4.70,
176,176,nador,0,MA,40,35.17,-2.93,75.20,6.93,
297,297,mogok,0,MM,51,22.92,96.51,72.24,2.77,
311,311,balkanabat,99,TM,41,39.51,54.36,79.80,6.69,
393,393,juiz de fora,20,BR,44,-21.76,-43.35,75.20,8.05,
443,443,lucapa,65,AO,48,-8.42,20.74,78.36,2.73,
444,444,birjand,0,IR,26,32.86,59.22,71.60,9.17,
468,468,san cristobal,75,EC,37,-0.39,-78.55,71.60,4.70,
534,534,marzuq,84,YE,36,14.40,46.47,79.44,6.02,


In [29]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City_ID,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,7,opuwo,0,NaN,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
153,153,porto novo,20,BR,53,-23.68,-45.44,77.00,4.70,Hotel Pousada Paradise
176,176,nador,0,MA,40,35.17,-2.93,75.20,6.93,Hotel Mediterranee
297,297,mogok,0,MM,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
311,311,balkanabat,99,TM,41,39.51,54.36,79.80,6.69,Nebitchi Hotel
393,393,juiz de fora,20,BR,44,-21.76,-43.35,75.20,8.05,Victory Business Hotel
443,443,lucapa,65,AO,48,-8.42,20.74,78.36,2.73,NaN
444,444,birjand,0,IR,26,32.86,59.22,71.60,9.17,هتل پرويز
468,468,san cristobal,75,EC,37,-0.39,-78.55,71.60,4.70,Hospedaje - Casa 929
534,534,marzuq,84,YE,36,14.40,46.47,79.44,6.02,NaN


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map


# Display figure
